In [15]:

from openai import OpenAI

base_url='http://localhost:8000/v1'
client = OpenAI(api_key='token-abc123', base_url=base_url)

gen_config = {
    'max_input_tokens': 10000,
    'max_tokens': 2048,
    'temperature': 0.3,
    'top_p': 0.6
}


llm_cfg = {
        # Use a model service compatible with the OpenAI API
        'model': client.models.list().data[0].id,
        'model_server': client.base_url.__str__(),  # base_url, also known as api_base
        'api_key': client.api_key,
        'generate_cfg': gen_config
    }


from qwen_agent.agents import TIRMathAgent

tir_prompt = """Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."""

tir_agent = TIRMathAgent(llm=llm_cfg, name="Qwen2.5-Math", system_message=tir_prompt)
question = "What is the integral of x^2 from 0 to 1?"
messages = [{'role': 'user', 'content': question}]
*_, last = tir_agent.run(messages, function_list=['code_interpreter'])

print(last[-1]['content'])


To find the integral of \(x^2\) from 0 to 1, we can use the fundamental theorem of calculus. The integral of \(x^2\) with respect to \(x\) is \(\frac{x^3}{3}\). Evaluating this from 0 to 1 gives us:

\[
\int_{0}^{1} x^2 \, dx = \left. \frac{x^3}{3} \right|_{0}^{1} = \frac{1^3}{3} - \frac{0^3}{3} = \frac{1}{3}
\]

Let's confirm this using Python and the sympy library.

```python
import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the function to integrate
f = x**2

# Compute the definite integral from 0 to 1
integral_result = sp.integrate(f, (x, 0, 1))

# Print the result
print(integral_result)
```
```output
1/3
```
The integral of \(x^2\) from 0 to 1 is \(\boxed{\frac{1}{3}}\).


接下来逐步展示以上过程的细节

In [16]:
import json
from typing import Tuple

from qwen_agent.tools.python_executor import PythonExecutor


def extract_program(result: str, last_only=True):
    """
    extract the program after "```python", and before "```"
    """
    program = ''
    start = False
    for line in result.split('\n'):
        if line.startswith('```python') or line.endswith('```python'):
            if last_only:
                program = ''  # only extract the last program
            else:
                program += '\n# ========\n'
            start = True
        elif line.startswith('```'):
            start = False
        elif start:
            program += line + '\n'
    if start:
        # the code is incomplete
        program = ''
    return program

def _detect_tool(text: str) -> Tuple[bool, str, str, str]:
    program = extract_program(text)
    if program:
        program = json.dumps({'code': program}, ensure_ascii=False)
    return (program != ''), PythonExecutor.name, program, text


检测及解析生成结果

In [20]:

messages = [{'role': 'system', 'content': tir_prompt}, {'role': 'user', 'content': question}]
for output in tir_agent._call_llm(messages=messages, stream=True):
    pass
    
has_action, action, action_input, thought = _detect_tool(output[-1]['content'])

In [ ]:
print(f"has Action: {has_action}")
print(f"Action: {action}")
print(f"Action Input: {action_input}")
print(f"Thought: {thought}")

has Action: True
Action: python_executor
Action Input: {"code": "import sympy as sp\n\n# Define the variable\nx = sp.symbols('x')\n\n# Define the function to integrate\nf = x**2\n\n# Compute the definite integral from 0 to 1\nintegral_value = sp.integrate(f, (x, 0, 1))\n\n# Print the result\nprint(integral_value)\n"}
Thought: To find the integral of \(x^2\) from 0 to 1, we can use the fundamental theorem of calculus. The integral of \(x^2\) with respect to \(x\) is \(\frac{x^3}{3}\). Evaluating this from 0 to 1 gives us:

\[
\int_{0}^{1} x^2 \, dx = \left. \frac{x^3}{3} \right|_{0}^{1} = \frac{1^3}{3} - \frac{0^3}{3} = \frac{1}{3}
\]

Let's confirm this using Python and the sympy library.

```python
import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the function to integrate
f = x**2

# Compute the definite integral from 0 to 1
integral_value = sp.integrate(f, (x, 0, 1))

# Print the result
print(integral_value)
```



工具执行

In [26]:
import json5

OBS_START = '```output'
OBS_END = '\n```\n'
observation = tir_agent._call_tool(action, action_input, messages=messages)
print(f"raw Observation: {observation}")

observation_list = json5.loads(observation)
if observation_list[-1] == 'Done':
    observation = observation_list[0]
else:
    observation = observation_list[-1]

observation = observation.strip()
observation = f'{OBS_START}\n{observation}{OBS_END}'

print(f"parsed Observation:\n {observation}")

raw Observation: [
    "1/3",
    "Done"
]
parsed Observation:
 ```output
1/3
```



组合输出

In [28]:
response = output[-1]['content']
if not response.endswith('\n'):
    response += '\n'
response += observation

new_messages = [{'role': 'system', 'content': tir_prompt}, {'role': 'user', 'content': response}]
for new_out in tir_agent._call_llm(messages=new_messages, stream=True):
    pass
print(f"New output:\n {new_out[-1]['content']}")

# The final answer is in the last message
response += new_out[-1]['content']
print(f"Final answer:\n {response}")

New output:
 The integral of \(x^2\) from 0 to 1 is \(\boxed{\frac{1}{3}}\). The Python code confirms that the result is correct.
Final answer:
 To find the integral of \(x^2\) from 0 to 1, we can use the fundamental theorem of calculus. The integral of \(x^2\) with respect to \(x\) is \(\frac{x^3}{3}\). Evaluating this from 0 to 1 gives us:

\[
\int_{0}^{1} x^2 \, dx = \left. \frac{x^3}{3} \right|_{0}^{1} = \frac{1^3}{3} - \frac{0^3}{3} = \frac{1}{3}
\]

Let's confirm this using Python and the sympy library.

```python
import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the function to integrate
f = x**2

# Compute the definite integral from 0 to 1
integral_value = sp.integrate(f, (x, 0, 1))

# Print the result
print(integral_value)
```
```output
1/3
```
The integral of \(x^2\) from 0 to 1 is \(\boxed{\frac{1}{3}}\). The Python code confirms that the result is correct.
